In [16]:
import sys
import os

# Get the project directory
project_path = "/home/diego/Git/thesis-tabtrans"

sys.path.append(project_path) #This helps to be able to import the data from the parent directory to other files

from utils import tabtrans_file, data
import openml
from sklearn import datasets, model_selection, pipeline, metrics
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler #to create one hot encoding for categorical variables
from collections import Counter
import numpy as np




In [17]:
id = 1484

if id in [1484,1564]: #two selected datasets with no task id, just id
    df = data.read_dataset_by_id(id)
else:
    task_id = id
    task = openml.tasks.get_task(task_id)
    dataset_id = task.dataset_id #suppose we input the task id 
    df = read_dataset_by_id(dataset_id)

X = df["features"] #features
y = df["outputs"].codes #outputs

label_counts = Counter(y) #counts how many instances of each class I have

labels_to_keep = {label for label, count in label_counts.items() if count > 10} #I will keep only the classes that have more than 25 instances

mask = np.isin(y, list(labels_to_keep))

#Filtering the classes with less than 10 instances
X_masked = X[mask]
y_masked = y[mask]

unique_values, counts = np.unique(y, return_counts=True)
n_labels = len(unique_values)

#create label encoder for the outputs just to have an order in values
encoder = LabelEncoder()

#encode the labels (converts strings to integers from 0 to n-1)
y = encoder.fit_transform(y)

#First I want to get the 80-20-20 split (Train-Test-Validation)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_masked, y_masked, test_size=0.20, random_state= 11, stratify = y_masked)
train_indices, val_indices = model_selection.train_test_split(np.arange(X_train.shape[0]), test_size=.33, random_state= 11, stratify = y_train) #.33 of train is equal to 20% of total


INFO:openml.datasets.dataset:pickle write lsvt


In [21]:
train_indices[0]

17

In [22]:
type(train_indices)

numpy.ndarray

In [23]:
new_train, _ = model_selection.train_test_split(train_indices, train_size=.80, random_state= 11, stratify = y_train[train_indices]) #.33 of train is equal to 20% of total
